# ALPS2

Evaluate and compare ALPS-based search methods on benchmark datasets.

**Methods**
- Original ALPS
- Myopic Method
- Section Method
- ALPS2


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path

# Make local modules importable (works whether Jupyter starts in repo root or this notebook's directory).
NOTEBOOK_DIR = Path.cwd()
REPO_ROOT = NOTEBOOK_DIR
while REPO_ROOT != REPO_ROOT.parent and not (REPO_ROOT / 'middleware').exists():
    REPO_ROOT = REPO_ROOT.parent

for p in [REPO_ROOT, REPO_ROOT / 'improved', NOTEBOOK_DIR]:
    ps = str(p)
    if ps not in sys.path:
        sys.path.insert(0, ps)

print('Repo root:', REPO_ROOT)
print('Notebook dir:', NOTEBOOK_DIR)


In [ ]:
import pickle

try:
    # Preferred import when the file is in improved/Functions_improved.py
    from improved.Functions_improved import (
        full_search_nk,
        myopic_search_nk,
        section_search_nk,
        hybrid_smart_search_nk,
    )
except Exception:
    # Fallback when running from inside improved/ directly
    from Functions_improved import (
        full_search_nk,
        myopic_search_nk,
        section_search_nk,
        hybrid_smart_search_nk,
    )

METHODS = {
    'Original ALPS': full_search_nk,
    'Myopic Method': myopic_search_nk,
    'Section Method': section_search_nk,
    'ALPS2': hybrid_smart_search_nk,
}

print('Loaded methods:', ', '.join(METHODS.keys()))

## Run
Select a method to run on all four datasets (ts1.p, ts2.p, ts3.p, ts4.p).

In [ ]:
import os
import time

# Define dataset paths
DATASETS = {
    'ts1': os.path.join(str(REPO_ROOT), 'ts1.p'),
    'ts2': os.path.join(str(REPO_ROOT), 'ts2.p'),
    'ts3': os.path.join(str(REPO_ROOT), 'ts3.p'),
    'ts4': os.path.join(str(REPO_ROOT), 'ts4.p'),
}

def run_single(method: str, dataset_name: str, dataset_path: str):
    """Run a single method on a single dataset"""
    fn = METHODS[method]
    
    # Load the pickle file
    try:
        with open(dataset_path, 'rb') as f:
            Data = pickle.load(f)
    except Exception as e:
        print(f'  ❌ Error loading {dataset_name}: {e}')
        return None
    
    # Run the method with default parameters (p=4, q=2) and time it
    try:
        start_time = time.time()
        result = fn(Data, p=4, q=2)
        elapsed_time = time.time() - start_time
        return result + [elapsed_time]  # [n, lambda, sigma, time]
    except Exception as e:
        print(f'  ❌ Error running {dataset_name}: {e}')
        return None

def run_all(method: str):
    """Run selected method on all datasets"""
    print(f'\n{"="*70}')
    print(f'Running: {method}')
    print(f'{"="*70}\n')
    
    results = {}
    total_time = 0
    
    for dataset_name, dataset_path in DATASETS.items():
        print(f'Processing {dataset_name}...')
        result = run_single(method, dataset_name, dataset_path)
        if result is not None:
            results[dataset_name] = result
            total_time += result[3]
            print(f'  ✓ {dataset_name}: n={result[0]}, λ={result[1]:.6f}, σ²={result[2]:.6f}, time={result[3]:.3f}s')
        print()
    
    print(f'{"="*70}')
    print(f'Summary for {method}:')
    print(f'{"="*70}')
    print(f'{"Dataset":<8} {"n":<6} {"λ":<12} {"σ²":<12} {"Time (s)":<10}')
    print(f'{"-"*70}')
    for dataset_name, result in results.items():
        print(f'{dataset_name:<8} {result[0]:<6} {result[1]:<12.6f} {result[2]:<12.6f} {result[3]:<10.3f}')
    print(f'{"-"*70}')
    print(f'{"TOTAL":<8} {"":<6} {"":<12} {"":<12} {total_time:<10.3f}')
    print(f'{"="*70}')
    
    return results

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display

    method_dd = widgets.Dropdown(
        options=list(METHODS.keys()),
        value='ALPS2',
        description='Method:',
        style={'description_width': 'initial'}
    )
    
    run_btn = widgets.Button(description='Run on All Datasets', button_style='primary')
    out = widgets.Output()

    def _on_click(_):
        out.clear_output()
        with out:
            try:
                results = run_all(method_dd.value)
            except Exception as e:
                print('Error:', repr(e))
                import traceback
                traceback.print_exc()

    run_btn.on_click(_on_click)
    
    # Display instructions
    print('ALPS2 Tool - Select a method to run on all datasets')
    print('Datasets: ts1.p, ts2.p, ts3.p, ts4.p')
    print('=' * 60)
    
    display(method_dd, run_btn, out)

except Exception:
    # Text fallback
    print('ipywidgets not available; using text input.')
    print('Methods:', ', '.join(METHODS.keys()))
    m = input('Method (exact): ').strip()
    run_all(m)